<a href="https://colab.research.google.com/github/rprimi/IR_bm25/blob/main/proj0_is_bm25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Projeto de construção de um sistema de recuperação de informação para seleção na disciplina da Unicamp "Deep Learning aplicado a sistemas de buscas"




1. Aprendendo sobre BM25 com o chatGPT do new Bing

Olá, este é o Bing. Eu posso te ajudar. 😊

BM25 é uma função de classificação usada por mecanismos de busca para estimar a relevância de documentos para uma determinada consulta de pesquisa¹². É baseada no modelo probabilístico de recuperação de informação desenvolvido nas décadas de 1970 e 1980 por Stephen E. Robertson, Karen Spärck Jones e outros².

BM25 é uma função que pontua um conjunto de documentos com base nos termos da consulta que aparecem em cada documento, independentemente da proximidade entre os termos dentro do documento². É uma família de funções de pontuação com componentes e parâmetros ligeiramente diferentes. Uma das instâncias mais proeminentes da função é a seguinte²:

$$
\mathrm{score}(D,Q) = \sum_{i=1}^{n} \mathrm{IDF}(q_i) \cdot \frac{f(q_i,D) \cdot (k_1 + 1)}{f(q_i,D) + k_1 \cdot (1 - b + b \cdot |D| / \mathrm{avgdl})}
$$


onde:

- `D` é um documento
- `Q` é uma consulta
- `f(q_i,D)` é a frequência do termo `q_i` no documento `D`
- `|D|` é o comprimento do documento `D` em palavras
- `avgdl` é o comprimento médio dos documentos na coleção
- `k_1` e `b` são parâmetros livres, geralmente escolhidos como `k_1 = 2,0` e `b = 0,75`
- `IDF(q_i)` é o peso do termo inverso da frequência do documento para o termo `q_i`, calculado como:

```
IDF(q_i) = log((N - n(q_i) + 0.5) / (n(q_i) + 0.5))
```

onde:

- `N` é o número total de documentos na coleção
- `n(q_i)` é o número de documentos que contêm o termo `q_i`

A ideia por trás do BM25 é que quanto mais vezes um termo da consulta aparece em um documento, maior será a relevância desse documento para a consulta. No entanto, essa relevância também depende da frequência do termo na coleção inteira (quanto mais raro for o termo, maior será seu peso), do comprimento do documento (documentos mais curtos tendem a ter pontuações mais altas) e dos parâmetros livres que podem ser ajustados para diferentes cenários.

Existem várias implementações em Python da função BM25 disponíveis online⁶⁷⁹ [^10^] ¹¹. Uma delas é a biblioteca python-bm25⁶, que pode ser instalada usando pip:


Origem: conversa com o Bing, 16/02/2023(1) Okapi BM25 - Wikipedia. https://en.wikipedia.org/wiki/Okapi_BM25 Acessado 16/02/2023.
(2) Okapi BM25 - Wikipedia. https://en.wikipedia.org/wiki/Okapi_BM25 Acessado 16/02/2023.
(3) python-bm25 | Python implementation of BM25 function | Search Engine .... https://kandi.openweaver.com/python/fanta-mnix/python-bm25#:~:text=python-bm25%20is%20a%20Python%20library%20typically%20used%20in,Python%20implementation%20of%20BM25%20function%20for%20document%20retrieval Acessado 16/02/2023.
(4) BM25 using Python Gensim Package | Search Engine. https://iamgeekydude.com/2022/12/25/bm25-using-python-gensim-package-search-engine-nlp/ Acessado 16/02/2023.
(5) GitHub - nhirakawa/BM25: A Python implementation of the BM25 ranking .... https://github.com/nhirakawa/BM25 Acessado 16/02/2023.
(6) GitHub - fanta-mnix/python-bm25: Python implementation of BM25 function .... https://github.com/fanta-mnix/python-bm25 Acessado 16/02/2023.
(7) GitHub - xianchen2/Text_Retrieval_BM25: Python implementation of the .... https://github.com/xianchen2/Text_Retrieval_BM25 Acessado 16/02/2023.
(8) Machine Learning Approach for Improved BM25 Retrieval. https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/LearningBM25MSRTechReport.pdf Acessado 16/02/2023.
(9) Information retrieval evaluation of precision, recall, f-score, AP, MAP .... https://stackoverflow.com/questions/40457331/information-retrieval-evaluation-of-precision-recall-f-score-ap-map-in-pytho Acessado 16/02/2023.
(10) Information Retrieval with document Re-ranking with BERT and BM25. https://medium.com/@papai143/information-retrieval-with-document-re-ranking-with-bert-and-bm25-7c29d738df73 Acessado 16/02/2023.
(11) python-bm25 | Python implementation of BM25 function | Search Engine .... https://kandi.openweaver.com/python/fanta-mnix/python-bm25 Acessado 16/02/2023.

In [ ]:

```
pip install python-bm25
```

Um exemplo simples de uso dessa biblioteca é o seguinte:

```python
# Importar a classe BM25Okapi
from bm25 import BM25Okapi

# Criar uma lista de documentos como listas de palavras
docs = [
    ["black", "cat", "white", "cat"],
    ["cat", "outer", "space"],
    ["wag", "dog"]
]

# Criar um objeto BM25Okapi com os documentos
bm25 = BM25Okapi(docs)

# Criar uma consulta como lista de palavras
query = ["cat"]

# Calcular as pontuações dos documentos para a consulta usando get_scores()
scores = bm25.get_scores(query)

# Imprimir as pontuações
print(scores)
```

A saída deste código será:

```
[2.8327298855090213, 2.8327298855090213, 0]
```
#  Isso significa que os dois primeiros documentos têm a mesma pontuação BM25 para a consulta "cat" e são mais relevantes do que o terceiro documento.

# Espero que isso tenha esclarecido o que é BM25 e como implementá-lo em Python.

Claro, eu posso explicar cada elemento da fórmula BM25.

A fórmula BM25 é a seguinte²:

$$
score(D,Q) = sum_{i=1}^{n} IDF(q_i) * (f(q_i,D) * (k_1 + 1)) / (f(q_i,D) + k_1 * (1 - b + b * |D| / avgdl))
$$


Os elementos da fórmula são:

- `score(D,Q)` é a pontuação BM25 do documento `D` para a consulta `Q`
- `sum_{i=1}^{n}` é a soma sobre todos os termos da consulta `q_i` que aparecem no documento `D`
- `IDF(q_i)` é o peso do termo inverso da frequência do documento para o termo `q_i`, que mede o quão informativo é o termo para a consulta. É calculado como:

$$IDF(q_i) = log((N - n(q_i) + 0.5) / (n(q_i) + 0.5))$$


onde:

- $N$ é o número total de documentos na coleção
- `n(q_i)` é o número de documentos que contêm o termo `q_i`

Quanto menor for o valor de `n(q_i)`, maior será o valor de `IDF(q_i)` e maior será a contribuição do termo para a pontuação final.

- `(f(q_i,D) * (k_1 + 1)) / (f(q_i,D) + k_1 * (1 - b + b * |D| / avgdl))` é um fator de normalização que leva em conta a frequência do termo no documento, o comprimento do documento e os parâmetros livres. É composto por:

  - `f(q_i,D)` é a frequência do termo `q_i` no documento `D`, ou seja, quantas vezes ele aparece no texto. Quanto maior for esse valor, maior será a relevância do documento para o termo.
  - `(k_1 + 1)` e `(k_1 * (1 - b + b * |D| / avgdl))` são dois componentes que ajustam a influência da frequência do termo na pontuação final. Eles dependem dos parâmetros livres:
    - `k_1` controla o quanto a pontuação aumenta com base na frequência do termo. Valores maiores de `k_1` significam que as diferenças na frequência do termo têm mais impacto na pontuação, enquanto valores menores significam que as diferenças têm menos impacto. O valor padrão geralmente usado é `k_1 = 2,0`.
    - `b` controla o quanto a pontuação é afetada pelo comprimento do documento em relação ao comprimento médio dos documentos na coleção. Valores maiores de `b` favorecem documentos mais curtos, enquanto valores menores favorecem documentos mais longos. O valor padrão geralmente usado é `b = 0,75`.
    - |D|é o comprimento do documento D em palavras
    - avgdlé o comprimento médio dos documentos na coleção

Esses elementos juntos formam uma função de classificação que tenta estimar quão relevante é um documento para uma consulta com base nos termos que eles compartilham.

Espero que isso tenha esclarecido os elementos da fórmula BM25.

Origem: conversa com o Bing, 16/02/2023(1) Okapi BM25 - Wikipedia. https://en.wikipedia.org/wiki/Okapi_BM25 Acessado 16/02/2023.
(2) Practical BM25 - Part 2: The BM25 Algorithm and its Variables. https://www.elastic.co/blog/practical-bm25-part-2-the-bm25-algorithm-and-its-variables Acessado 16/02/2023.
(3) Document similarities with BM25 algorithm - MATLAB bm25Similarity. https://www.mathworks.com/help/textanalytics/ref/bm25similarity.html Acessado 16/02/2023.